In [12]:
import torch
import numpy as np
import random
from types import SimpleNamespace as Namespace
from config import config_dict as cfg
cfg = Namespace(**cfg)
from model_bert import TSCModel_PL
from train_apply import load_csvs, load_data
from inference import load_data_for_inference
from transformers import BertTokenizer
from captum.attr import IntegratedGradients, LayerIntegratedGradients
from captum.attr import visualization as viz
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True

In [13]:
def get_toxic_examples(df):
    return df[df.iloc[:, 2:].sum(axis=1) > 0]

In [14]:
# adapted from Captum Tutorial: https://captum.ai/tutorials/Bert_SQUAD_Interpret
def construct_input_ref_pair(text, tokenizer, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False, truncation=False, max_length=cfg.truncate_seq_len)

    # construct input ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    input_ids = torch.tensor([input_ids], device=device, dtype=torch.int)
    input_token_type_ids = torch.zeros_like(input_ids, device=device, dtype=torch.int)
    input_attention_mask = torch.ones_like(input_ids, device=device, dtype=torch.int)

    # construct ref ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id] 
    ref_input_ids = torch.tensor([ref_input_ids], device=device, dtype=torch.int)
    ref_token_type_ids = torch.zeros_like(ref_input_ids, device=device, dtype=torch.int)
    ref_attention_mask = torch.ones_like(input_ids, device=device, dtype=torch.int)

    return {"input_ids": input_ids, "token_type_ids": input_token_type_ids, "attention_mask": input_attention_mask}, \
        {"input_ids": ref_input_ids, "token_type_ids": ref_token_type_ids, "attention_mask": ref_attention_mask}



In [15]:
def predict(model, input_dict):
    pred = model(input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])
    pred = torch.sigmoid(pred)
    return pred

def attribute(lig:LayerIntegratedGradients, input_dict, ref_dict, labels):
    res_attr, res_delta = [], []
    for target in range(6):
        attributions, delta = lig.attribute(inputs=input_dict["input_ids"], baselines=ref_dict["input_ids"], \
                                        additional_forward_args=(input_dict["token_type_ids"], input_dict["attention_mask"]), \
                                            target = target,
                                            return_convergence_delta=True, n_steps=50)
        attributions_sum = attributions.sum(dim=-1).squeeze(0)
        res_attr.append((attributions_sum.detach().cpu()))
        res_delta.append((delta.detach().cpu()))
    return torch.stack(res_attr), torch.stack(res_delta)

def visualize(attributions, deltas, predictions, raw_tokens, labels):
    print(raw_tokens)
    for target in range(6):
        print(f"Target: {cfg.label_tags[target]}")
        attribution = attributions[target]
        prediction = predictions[target]
        pred_vis = viz.VisualizationDataRecord(
                                word_attributions=attribution,
                                pred_prob = prediction,
                                pred_class=int(prediction>0.5),
                                true_class=labels[target],
                                attr_class=int(prediction>0.5),
                                attr_score=attribution.sum(),
                                raw_input_ids=raw_tokens,
                                convergence_score=deltas[target],
    )
        viz.visualize_text([pred_vis])

In [16]:

checkpoint_path = "/home/david/TU-Kurse/WS22_23/PML/BERT-TSC/lightning_logs/cluster/version_22/checkpoints/epoch=1-step=9974.ckpt"
checkpoint = torch.load(checkpoint_path, map_location=device)
pretrained_state_dict = checkpoint["state_dict"]
model = TSCModel_PL(cfg)
model.load_pretrained_weights_whole_model(pretrained_state_dict)
model = model.to(device)
tokenizer : BertTokenizer = BertTokenizer.from_pretrained("bert-base-cased")


In [17]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [18]:
train, test = load_csvs()

In [19]:

toxic_train = get_toxic_examples(train)
toxic_test = get_toxic_examples(test)

example1 = toxic_train.iloc[7]
sentence1 = example1["comment_text"]
labels1 = example1.iloc[2:].to_list()
print(sentence1)
print(labels1)

example2 = toxic_test.iloc[1]
sentence2 = example2["comment_text"]
labels2 = example2.iloc[2:].to_list()
print(sentence2)
print(labels2)


Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell!
[1, 1, 1, 0, 1, 0]
DJ Robinson is gay as hell! he sucks his dick so much!!!!!
[1, 0, 1, 0, 1, 1]


In [20]:

lig = LayerIntegratedGradients(model, model.backbone.embeddings)
examples = toxic_train.iloc[0:10]
for index, example in examples.iterrows():
    sentence = example["comment_text"]
    labels = example.iloc[2:].to_list()

    input_dict, ref_dict = construct_input_ref_pair(sentence, tokenizer, ref_token_id, sep_token_id, cls_token_id)
    raw_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].tolist()[0])
    # input_dict_tok = tokenizer(sentence1, return_tensors="pt", padding=True, truncation=True, return_token_type_ids=True, return_attention_mask=True)
    with torch.no_grad():
        attributions, deltas = attribute(lig, input_dict, ref_dict, labels)
        predictions = predict(model, input_dict)[0].detach().cpu()
    visualize(attributions, deltas, predictions, raw_tokens, labels)
    del(attributions, deltas, predictions, raw_tokens, input_dict, ref_dict)



['[CLS]', 'CO', '##C', '##KS', '##UC', '##KE', '##R', 'B', '##EF', '##OR', '##E', 'YOU', 'P', '##IS', '##S', 'AR', '##O', '##UN', '##D', 'ON', 'M', '##Y', 'W', '##OR', '##K', '[SEP]']
Target: toxic


Target: severe_toxic


Target: obscene


Target: threat


Target: insult


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.79),1,10.55,[CLS] = Tony Sid ##away is obviously a fist ##fu ##cke ##e . He loves an arm up his ass . [SEP]


Target: identity_hate


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.10),0,6.54,[CLS] = Tony Sid ##away is obviously a fist ##fu ##cke ##e . He loves an arm up his ass . [SEP]


: 

: 